<a href="https://colab.research.google.com/github/massinoLight/tp_AlgoBD/blob/main/TP_pyspark_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 198 kB 46.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=e582b01fcbeeb695101a9c49a80e43e08351556430965a1ba7715ba56bfc09bc
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


**On lit la BD avec panda car la librérie pyspark pour le read csv nest pas stable**

In [ ]:
import pandas as pd

df=pd.read_csv('/content/sample_data/iris.csv')
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


#load data as Spark DataFrame
data2=spark.read.format("csv") \
    .option("header","true") \
    .option("mode","DROPMALFORMED") \
    .load('C:\\Users\\Dell\\Desktop\\iris.csv')

##On transforme la df panda en rdd pyspark 

In [ ]:
from pyspark.sql import SparkSession
spark= SparkSession.builder \
    .master("local") \
    .appName("Data Exploration") \
    .getOrCreate()

irisSparkDF=spark.createDataFrame(df)
irisSparkDF.printSchema()
irisSparkDF.show() 


    



root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- class: string (nullable = true)

+------------+-----------+------------+-----------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|      class|
+------------+-----------+------------+-----------+-----------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|
|         4.9| 

##on passe la rdd au bon format features label

In [ ]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

trainingData=irisSparkDF.rdd.map(lambda x:(Vectors.dense(x[0:-1]), x[-1])).toDF(["features", "label"])
trainingData.show()
# +-----------------

+-----------------+-----------+
|         features|      label|
+-----------------+-----------+
|[5.1,3.5,1.4,0.2]|Iris-setosa|
|[4.9,3.0,1.4,0.2]|Iris-setosa|
|[4.7,3.2,1.3,0.2]|Iris-setosa|
|[4.6,3.1,1.5,0.2]|Iris-setosa|
|[5.0,3.6,1.4,0.2]|Iris-setosa|
|[5.4,3.9,1.7,0.4]|Iris-setosa|
|[4.6,3.4,1.4,0.3]|Iris-setosa|
|[5.0,3.4,1.5,0.2]|Iris-setosa|
|[4.4,2.9,1.4,0.2]|Iris-setosa|
|[4.9,3.1,1.5,0.1]|Iris-setosa|
|[5.4,3.7,1.5,0.2]|Iris-setosa|
|[4.8,3.4,1.6,0.2]|Iris-setosa|
|[4.8,3.0,1.4,0.1]|Iris-setosa|
|[4.3,3.0,1.1,0.1]|Iris-setosa|
|[5.8,4.0,1.2,0.2]|Iris-setosa|
|[5.7,4.4,1.5,0.4]|Iris-setosa|
|[5.4,3.9,1.3,0.4]|Iris-setosa|
|[5.1,3.5,1.4,0.3]|Iris-setosa|
|[5.7,3.8,1.7,0.3]|Iris-setosa|
|[5.1,3.8,1.5,0.3]|Iris-setosa|
+-----------------+-----------+
only showing top 20 rows



##On applique l'ACP (2 features)

In [ ]:
from pyspark.ml.feature import PCA as PCAml
from pyspark.ml.linalg import Vectors 



pca = PCAml(k=2, inputCol="features", outputCol="pca")
model = pca.fit(trainingData)
transformed = model.transform(trainingData)

In [ ]:
transformed.show()

+-----------------+-----------+--------------------+
|         features|      label|                 pca|
+-----------------+-----------+--------------------+
|[5.1,3.5,1.4,0.2]|Iris-setosa|[-2.8271359726790...|
|[4.9,3.0,1.4,0.2]|Iris-setosa|[-2.7959524821488...|
|[4.7,3.2,1.3,0.2]|Iris-setosa|[-2.6215235581650...|
|[4.6,3.1,1.5,0.2]|Iris-setosa|[-2.7649059004742...|
|[5.0,3.6,1.4,0.2]|Iris-setosa|[-2.7827501159516...|
|[5.4,3.9,1.7,0.4]|Iris-setosa|[-3.2314457367733...|
|[4.6,3.4,1.4,0.3]|Iris-setosa|[-2.6904524156023...|
|[5.0,3.4,1.5,0.2]|Iris-setosa|[-2.8848611044591...|
|[4.4,2.9,1.4,0.2]|Iris-setosa|[-2.6233845324473...|
|[4.9,3.1,1.5,0.1]|Iris-setosa|[-2.8374984110638...|
|[5.4,3.7,1.5,0.2]|Iris-setosa|[-3.0048163084440...|
|[4.8,3.4,1.6,0.2]|Iris-setosa|[-2.8982003795119...|
|[4.8,3.0,1.4,0.1]|Iris-setosa|[-2.7239091217858...|
|[4.3,3.0,1.1,0.1]|Iris-setosa|[-2.2861426515079...|
|[5.8,4.0,1.2,0.2]|Iris-setosa|[-2.8677998808418...|
|[5.7,4.4,1.5,0.4]|Iris-setosa|[-3.12747377398